In [1]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
from pennylane import BasisState, FermionicSingleExcitation, FermionicDoubleExcitation
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2

    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1



In [2]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100, p=0.0):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p, wires=w)
                qml.PhaseFlip(p, wires=w)
                qml.DepolarizingChannel(p, wires=w)
                qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0, p=0.0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200, p=0.02)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params, p=0.02)
#eigval.append(eig)
print("p=0.02 Bit-Flip Noise Model")
print('eigenvalues:', eig)

/Users/prince.kwao/miniconda3/envs/cwp/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:211: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


Ground state energy: -1.73597446 Ha
p=0.02 Bit-Flip Noise Model
eigenvalues: [tensor([-1.91636694, -1.91146531, -1.73706901, -1.69998303, -1.69776225,
        -1.66063018, -1.63777532, -1.43146713, -1.43081045, -1.42337176,
        -1.42266794, -1.31964345, -1.28017809, -1.27918194, -1.2333454 ,
        -1.22060944, -1.21120615, -1.12297919, -1.118343  , -1.02264934,
        -1.00713053, -0.91013867, -0.91005855, -0.7918367 , -0.78471462,
        -0.78325518, -0.73897813], requires_grad=True)]


In [3]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100, p=0.0):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p, wires=w)
                qml.PhaseFlip(p, wires=w)
                qml.DepolarizingChannel(p, wires=w)
                qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0, p=0.0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200, p=0.04)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params, p=0.04)
#eigval.append(eig)
print("p=0.04 Bit-Flip Noise Model")
print('eigenvalues:', eig)

Ground state energy: -1.60360665 Ha
p=0.04 Bit-Flip Noise Model
eigenvalues: [tensor([-1.92944238, -1.91889235, -1.74459817, -1.71885918, -1.71587182,
        -1.68425086, -1.63550777, -1.43173541, -1.43084521, -1.42414761,
        -1.42248344, -1.34416292, -1.27622635, -1.27429767, -1.23331884,
        -1.22001606, -1.21120565, -1.12297261, -1.11834055, -1.02526589,
        -0.99569881, -0.91010028, -0.9100907 , -0.77132298, -0.76911642,
        -0.76839147, -0.73966503], requires_grad=True)]


In [4]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100, p=0.0):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p, wires=w)
                qml.PhaseFlip(p, wires=w)
                qml.DepolarizingChannel(p, wires=w)
                qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0, p=0.0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200, p=0.06)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params, p=0.06)
#eigval.append(eig)
print("p=0.06 Bit-Flip Noise Model")
print('eigenvalues:', eig)

Ground state energy: -1.49619275 Ha
p=0.06 Bit-Flip Noise Model
eigenvalues: [tensor([-1.93391681, -1.92069292, -1.74883394, -1.73126393, -1.72795589,
        -1.69541168, -1.63205501, -1.43117767, -1.4300022 , -1.42399653,
        -1.4216746 , -1.35531624, -1.27316771, -1.27057083, -1.23331693,
        -1.21973713, -1.21120553, -1.1229639 , -1.11834043, -1.02711465,
        -0.98676208, -0.91045179, -0.91024909, -0.76105654, -0.7582643 ,
        -0.75011717, -0.73997407], requires_grad=True)]


In [5]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100, p=0.0):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p, wires=w)
                qml.PhaseFlip(p, wires=w)
                qml.DepolarizingChannel(p, wires=w)
                qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0, p=0.0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200, p=0.08)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params, p=0.08)
#eigval.append(eig)
print("p=0.08 Bit-Flip Noise Model")
print('eigenvalues:', eig)

Ground state energy: -1.40230099 Ha
p=0.08 Bit-Flip Noise Model
eigenvalues: [tensor([-1.93222189, -1.89314783, -1.77161472, -1.74392707, -1.73770759,
        -1.70040947, -1.63162903, -1.43044261, -1.4287557 , -1.42317409,
        -1.42037957, -1.33062143, -1.26722255, -1.26217536, -1.25771625,
        -1.23329708, -1.21119329, -1.12200259, -1.1183477 , -1.01684349,
        -0.94590221, -0.94183545, -0.88564178, -0.75263075, -0.72909825,
        -0.72171527, -0.71726819], requires_grad=True)]


In [6]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100, p=0.0):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p, wires=w)
                qml.PhaseFlip(p, wires=w)
                qml.DepolarizingChannel(p, wires=w)
                qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0, p=0.0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #for w in wires:
            #qml.BitFlip(p, wires=w)
            #qml.PhaseFlip(p, wires=w)
            #qml.DepolarizingChannel(p, wires=w)
            #qml.AmplitudeDamping(p, wires=w)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

active_electrons = 4
active_orbitals = 4
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200, p=0.1)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params, p=0.1)
#eigval.append(eig)
print("p=0.1 Bit-Flip Noise Model")
print('eigenvalues:', eig)

Ground state energy: -1.31599601 Ha
p=0.1 Bit-Flip Noise Model
eigenvalues: [tensor([-1.95321299, -1.90713459, -1.82463099, -1.79845808, -1.71630418,
        -1.68268828, -1.62884004, -1.43028354, -1.42791775, -1.42315875,
        -1.42020075, -1.32950925, -1.29877818, -1.29808162, -1.22387181,
        -1.21848804, -1.20840276, -1.16825497, -1.12165277, -1.11777219,
        -1.10279545, -1.03660693, -0.90983389, -0.90217042, -0.6592295 ,
        -0.47077329, -0.45147792], requires_grad=True)]
